In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score
from sklearn.datasets import fetch_openml
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from skopt import BayesSearchCV
from sklearn import preprocessing
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.multioutput import ClassifierChain

from scikeras.wrappers import KerasClassifier

In [3]:
x = np.load('../Part1.npy')
Y_t = np.load('../train.npy/y_train.npy')
#X_train = (x-np.min(x))/(np.max(x)-np.min(x))
xt = np.load('../Part2.npy')
Y_te = np.load('../test.npy/Y_Test.npy')
#X_test = (xt-np.min(xt))/(np.max(xt)-np.min(xt))
x_add = np.load('../Part3.npy')
y_add = np.load('../additional.npy/Y_additional.npy')
#x_add = (x_add-np.min(x_add))/(np.max(x_add)-np.min(x_add))
Y = np.concatenate((Y_t, Y_te, y_add))
X = np.concatenate((x, xt, x_add))
#X_train = (X_train-np.min(X_train))/(np.max(X_train)-np.min(X_train))
scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [49]:
Y_train[:,0].shape

(2155,)

In [47]:
def create_model(activation, filters, kernel_size, pool_size, hidden_units, dropout_rate):
    model = Sequential()
    model.add(Conv1D(8, kernel_size=kernel_size, activation=activation, input_shape=(None, 17)))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Dense(hidden_units, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(2, activation='sigmoid'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [42]:
class ChainCNN(BaseEstimator, ClassifierMixin):
    def __init__(self, activation, filters, kernel_size, pool_size, hidden_units, dropout_rate):
        super(ChainCNN, self).__init__()
        self.activation = activation
        self.filters = filters
        self.model = KerasClassifier(build_fn=create_model(activation, filters, kernel_size, pool_size, hidden_units, dropout_rate), verbose=0)
        self.chain = ClassifierChain(self.model, order="random", random_state=1)
        self.kernel_size = kernel_size
        self.pool_size = pool_size
        self.hidden_units = hidden_units
        self.dropout_rate = dropout_rate
    def fit(self, X_train, Y_train):
        return self.chain.fit(X_train, Y_train)
    
    def score(self, X_test, Y_test):
        Y_pred_chain = self.chain.predict(X_test)
        precision = precision_score(Y_test, Y_pred_chain>=0.5, average='samples')
        return precision

In [43]:
import warnings
warnings.filterwarnings("ignore")
search_space = {
    'activation': ['relu', 'sigmoid', 'tanh'],
    'filters': (16, 32),
    'kernel_size': (3, 5),
    'pool_size': (2, 3),
    'hidden_units': (64, 256),
    'dropout_rate': (0.2,0.5)
}
model = ChainCNN('relu', 16, 3,2,64,0.2)
opt = BayesSearchCV(estimator=model, search_spaces=search_space, n_jobs=-1, cv=7)
opt.fit(X_train, Y_train)


INFO:tensorflow:Assets written to: C:\Users\jonat\AppData\Local\Temp\tmp1yrmosvc\assets


INFO:tensorflow:Assets written to: C:\Users\jonat\AppData\Local\Temp\tmp1yrmosvc\assets


INFO:tensorflow:Assets written to: C:\Users\jonat\AppData\Local\Temp\tmptp4rdlq5\assets


INFO:tensorflow:Assets written to: C:\Users\jonat\AppData\Local\Temp\tmptp4rdlq5\assets


INFO:tensorflow:Assets written to: C:\Users\jonat\AppData\Local\Temp\tmpl_scwoz0\assets


...

INFO:tensorflow:Assets written to: C:\Users\jonat\AppData\Local\Temp\tmp_gwwlnsc\assets


INFO:tensorflow:Assets written to: C:\Users\jonat\AppData\Local\Temp\tmp_gwwlnsc\assets


INFO:tensorflow:Assets written to: C:\Users\jonat\AppData\Local\Temp\tmpvkad043k\assets


INFO:tensorflow:Assets written to: C:\Users\jonat\AppData\Local\Temp\tmpvkad043k\assets


ValueError: in user code:

    File "C:\Users\jonat\Documents\GraphAnalytics\gaenv\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\jonat\Documents\GraphAnalytics\gaenv\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\jonat\Documents\GraphAnalytics\gaenv\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\jonat\Documents\GraphAnalytics\gaenv\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\jonat\Documents\GraphAnalytics\gaenv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\jonat\Documents\GraphAnalytics\gaenv\lib\site-pac

In [ ]:
print("Best parameters found: ", opt.best_params_)
print("Best score: ", opt.best_score_)

In [ ]:
# Perform hyperparameter optimization
bayes_cv.fit(X_train, Y_train)

# Evaluate the best model on the test set
y_pred = bayes_cv.predict(X_test)

In [45]:
cnn_model = create_model('relu', 8, 3, 2, 64, 0.2)
cnn_model.fit(X_train, Y_train)

ValueError: in user code:

    File "C:\Users\jonat\Documents\GraphAnalytics\gaenv\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\jonat\Documents\GraphAnalytics\gaenv\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\jonat\Documents\GraphAnalytics\gaenv\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\jonat\Documents\GraphAnalytics\gaenv\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\jonat\Documents\GraphAnalytics\gaenv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\jonat\Documents\GraphAnalytics\gaenv\lib\site-pac

In [48]:
# Define the model
model = Sequential()

# Add a 1D convolutional layer
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(17, 1)))

# Add a max pooling layer
model.add(MaxPooling1D(pool_size=2))

# Flatten the output
model.add(Flatten())

# Add a fully connected layer
model.add(Dense(64, activation='relu'))

# Add the output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_57"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_57 (Conv1D)          (None, 15, 32)            128       
                                                                 
 max_pooling1d_51 (MaxPoolin  (None, 7, 32)            0         
 g1D)                                                            
                                                                 
 flatten_40 (Flatten)        (None, 224)               0         
                                                                 
 dense_101 (Dense)           (None, 64)                14400     
                                                                 
 dense_102 (Dense)           (None, 1)                 65        
                                                                 
Total params: 14,593
Trainable params: 14,593
Non-trainable params: 0
_________________________________________________

In [64]:
model.fit(X_train, Y_train[:,7])

68/68 [==============================] - 0s 5ms/step - loss: 0.3474 - accuracy: 0.8868


In [78]:
pred = model.predict(X_test)
#model.score(Y_test[:,7], pred)

23/23 [==============================] - 0s 3ms/step


In [76]:
precision_score(Y_test[:,7], np.where(pred>0.3,1,0))

0.3333333333333333